# GeoPandas

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giswqs/geog-312/blob/main/book/geospatial/geopandas.ipynb)

## Overview

[GeoPandas](https://geopandas.org) is an open-source Python library that simplifies working with geospatial data by extending Pandas data structures. It seamlessly integrates geospatial operations with a pandas-like interface, allowing for the manipulation of geometric types such as points, lines, and polygons. GeoPandas combines the functionalities of Pandas and Shapely, enabling geospatial operations like spatial joins, buffering, intersections, and projections with ease.

## Learning Objectives

By the end of this lecture, you should be able to:

- Understand the basic data structures in GeoPandas: `GeoDataFrame` and `GeoSeries`.
- Create `GeoDataFrames` from tabular data and geometric shapes.
- Read and write geospatial data formats like Shapefile and GeoJSON.
- Perform common geospatial operations such as measuring areas, distances, and spatial relationships.
- Visualize geospatial data using Matplotlib and GeoPandas' built-in plotting functions.
- Work with different Coordinate Reference Systems (CRS) and project geospatial data.

## Concepts

The core data structures in GeoPandas are `GeoDataFrame` and `GeoSeries`. A `GeoDataFrame` extends the functionality of a Pandas DataFrame by adding a geometry column, allowing spatial data operations on geometric shapes. The `GeoSeries` handles geometric data (points, polygons, etc.).

A `GeoDataFrame` can have multiple geometry columns, but only one is considered the active geometry at any time. All spatial operations are applied to this active geometry, accessible via the `.geometry` attribute.

## Installing and Importing GeoPandas

Before we begin, make sure you have geopandas installed. You can install it using:

In [ ]:
# %pip install geopandas

Once installed, import GeoPandas and other necessary libraries:

In [4]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

## 1. Creating GeoDataFrames

A GeoDataFrame is a tabular data structure that contains a `geometry` column, which holds the geometric shapes. You can create a GeoDataFrame from a list of geometries or from a pandas DataFrame.

In [ ]:
# Creating a GeoDataFrame from scratch
data = {
    "City": ["Tokyo", "New York", "London", "Paris"],
    "Latitude": [35.6895, 40.7128, 51.5074, 48.8566],
    "Longitude": [139.6917, -74.0060, -0.1278, 2.3522],
}

df = pd.DataFrame(data)
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))
gdf

## 2. Reading and Writing Geospatial Data

GeoPandas allows reading and writing a variety of geospatial formats, such as Shapefiles, GeoJSON, and more. We'll use a GeoJSON dataset of New York City borough boundaries.

<h3 style="color: orange;">Reading a GeoJSON File</h3>

We'll load the New York boroughs dataset from a GeoJSON file hosted online.

In [ ]:
url = "https://github.com/opengeos/datasets/releases/download/vector/nybb.geojson"
gdf = gpd.read_file(url)
gdf.head()

This `GeoDataFrame` contains several columns, including `BoroName`, which represents the names of the boroughs, and `geometry`, which stores the polygons for each borough.

<h3 style="color: orange;">Writing to a GeoJSON File</h3>

GeoPandas also supports saving geospatial data back to disk. For example, we can save the GeoDataFrame as a new GeoJSON file:

In [ ]:
output_file = "nyc_boroughs.geojson"
gdf.to_file(output_file, driver="GeoJSON")
print(f"GeoDataFrame has been written to {output_file}")

Similarly, you can write GeoDataFrames to other formats, such as Shapefiles, GeoPackage, and more.

In [ ]:
output_file = "nyc_boroughs.shp"
gdf.to_file(output_file)

In [15]:
output_file = "nyc_boroughs.gpkg"  # !much more efficient use this format
gdf.to_file(output_file, driver="GPKG")

### Downloading from internet

In [48]:
import leafmap 

In [ ]:
leafmap.download_file(url, output = "/home/felwind14/projects/py4geo/book/geospatial")

## 3. Simple Accessors and Methods

Now that we have the data, let's explore some simple GeoPandas methods to manipulate and analyze the geometric data.

### Measuring Area

We can calculate the area of each borough. GeoPandas automatically calculates the area of each polygon:

In [ ]:
gdf

In [ ]:
# Set BoroName as the index for easier reference
# ! Remember to update gdf to the new GeoDataFrame
gdf = gdf.set_index("BoroName")

# Calculate the area
gdf["area"] = gdf.area
gdf

### Getting Polygon Boundaries and Centroids

To get the boundary (lines) and centroid (center point) of each polygon:

In [ ]:
# Get the boundary of each polygon
gdf["boundary"] = gdf.boundary

# Get the centroid of each polygon
gdf["centroid"] = gdf.centroid

gdf[["boundary", "centroid"]]

In [ ]:
#Changing name to centroid column
gdf = gdf.rename(columns={"centroid": "centroid_boro"})
gdf

### Measuring Distance and filtering

We can also measure the distance from each borough's centroid to a reference point, such as the centroid of Manhattan.

In [ ]:
gdf.loc["Manhattan"]

In [ ]:
# Use Manhattan's centroid as the reference point
# ! Loc helps to select a specific row by its index value

manhattan_centroid = gdf.loc["Manhattan", "centroid_boro"]  #, "centroid_boro" to get the specific column

# Calculate the distance from each centroid to Manhattan's centroid
gdf["distance_to_manhattan"] = gdf["centroid_boro"].distance(manhattan_centroid)
gdf[["centroid_boro", "distance_to_manhattan"]]

In [ ]:
manhattan_geom = gdf.loc["Manhattan", "geometry"]  
manhattan_geom

#Note: it is able to find the geometry of Manhattan because we set the index to BoroName

### Calculating Mean Distance

We can calculate the mean distance between the borough centroids and Manhattan:

In [ ]:
mean_distance = gdf["distance_to_manhattan"].mean()
print(f"Mean distance to Manhattan: {mean_distance} units")

In [58]:
#We have to drop some columns to avoid having multiple geometries in the GeoDataFrame
gdf.drop(columns= ["boundary", "centroid_boro"], inplace=True) #inplace is similar gdf = gdf.operation


In [ ]:
gdf

In [59]:
gdf.to_file("nyc_dst_manhattan.geojson")

## 4. Plotting Geospatial Data

GeoPandas integrates with Matplotlib for easy plotting of geospatial data. Let's create some maps to visualize the data.

### Plotting the Area of Each Borough

We can color the boroughs based on their area and display a legend:

In [ ]:
gdf.columns

In [ ]:
gdf.plot()

In [ ]:
gdf.plot("area", legend=True, figsize=(10, 3))
plt.title("NYC Boroughs by Area")
plt.show()

### Plotting Centroids and Boundaries

We can also plot the centroids and boundaries:

In [ ]:
#Some fo the columns do not work once all the code is run because they are dropped in the previous cell
#we need to run the lines #gdf.boundary gdf.centroid and rename the centroid

gdf.columns

In [ ]:
# Plot the boundaries and centroids
ax = gdf["geometry"].plot(figsize=(10, 4), edgecolor="black")
gdf["centroid_boro"].plot(ax=ax, color="red", markersize=50)
plt.title("NYC Borough Boundaries and Centroids")
plt.show()

You can also explore your data interactively using `GeoDataFrame.explore()`, which behaves in the same way `plot()` does but returns an interactive map instead.

In [ ]:
gdf.explore("area", legend=False)

## Geometry Manipulations

GeoPandas provides several methods for manipulating geometries, such as buffering (creating a buffer zone around geometries) and computing convex hulls (the smallest convex shape enclosing the geometries).

### Buffering Geometries

We can create a buffer zone around each borough:

In [ ]:
# Buffer the boroughs by 10000 feet
gdf["buffered"] = gdf.buffer(10000)

# Plot the buffered geometries
gdf["buffered"].plot(alpha=0.5, edgecolor="black")  #alpha for opacity
plt.title("Buffered NYC Boroughs (10,000 feet)")
plt.show()


In [ ]:
gdf

### Convex Hulls

The convex hull is the smallest convex shape that can enclose a geometry. Let's calculate the convex hull for each borough:

In [ ]:
# Calculate convex hull
gdf["convex_hull"] = gdf.convex_hull  #there are multiple operations that can be done with the GeoDataFrame

# Plot the convex hulls
gdf["convex_hull"].plot(alpha=0.5, color="lightblue", edgecolor="black")
plt.title("Convex Hull of NYC Boroughs")
plt.show()

## Spatial Queries and Relations

We can also perform spatial queries to examine relationships between geometries. For instance, we can check which boroughs are within a certain distance of Manhattan.

### Checking for Intersections

We can find which boroughs' buffered areas intersect with the original geometry of Manhattan:

In [ ]:
# Get the geometry of Manhattan
manhattan_geom = gdf.loc["Manhattan", "geometry"]

# Check which buffered boroughs intersect with Manhattan's geometry
gdf["intersects_manhattan"] = gdf["buffered"].intersects(manhattan_geom)
gdf[["intersects_manhattan"]]

### Checking for Containment

Similarly, we can check if the centroids are contained within the borough boundaries:

In [ ]:
# Check if centroids are within the original borough geometries
gdf["centroid_within_borough"] = gdf["centroid_boro"].within(gdf["geometry"])
gdf[["centroid_within_borough"]]

## Projections and Coordinate Reference Systems (CRS)

GeoPandas makes it easy to manage projections. Each GeoSeries and GeoDataFrame has a crs attribute that defines its CRS.

### Checking the CRS

Let's check the CRS of the boroughs dataset:

In [ ]:
print(gdf.crs)

The CRS for this dataset is [`EPSG:2263`](https://epsg.io/2263) (NAD83 / New York State Plane). We can reproject the geometries to WGS84 ([`EPSG:4326`](https://epsg.io/4326)), which uses latitude and longitude coordinates.

[EPSG](https://epsg.io) stands for European Petroleum Survey Group, which was a scientific organization that standardized geodetic and coordinate reference systems. EPSG codes are unique identifiers that represent coordinate systems and other geodetic properties. 

### Reprojecting to WGS84

In [95]:
gdf.crs

In [ ]:
# Reproject the GeoDataFrame to WGS84 (EPSG:4326)
gdf_4326 = gdf.to_crs(epsg=4326)

# Plot the reprojected geometries
gdf_4326.plot(figsize=(10, 6), edgecolor="black")
plt.title("NYC Boroughs in WGS84 (EPSG:4326)")
plt.show()

Notice how the coordinates have changed from feet to degrees.

## Exercises

1. Create a GeoDataFrame containing a list of countries and their capital cities. Add a geometry column with the locations of the capitals.
2. Load a shapefile of your choice, filter the data to only include a specific region or country, and save the filtered GeoDataFrame to a new file.
3. Perform a spatial join between two GeoDataFrames: one containing polygons (e.g., country borders) and one containing points (e.g., cities). Find out which points fall within which polygons.
4. Plot a map showing the distribution of a particular attribute (e.g., population) across different regions.

In [ ]:
''# ? Exercise 1 - Create a GeoDataFrame from scratch
import geopandas as gpd

data = {
    "City": ["Tokyo", "New York", "London", "Paris"],
    "Country": ["Japan", "USA", "UK", "France"],
    "Latitude": [35.6895, 40.7128, 51.5074, 48.8566],
    "Longitude": [139.6917, -74.0060, -0.1278, 2.3522],
}

gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data["Longitude"], data["Latitude"]))
gdf

In [ ]:
x = gdf.crs
print(x)

In [ ]:
#assigning coordinate system

gdf.crs = "EPSG:4326"
gdf.crs

In [ ]:
gdf_3857 = gdf.to_crs(epsg=3857)
gdf_3857.crs
gdf_3857

In [ ]:
''# ? Exercise 2 - Load a GeoDataFrame from a file

import geopandas as gpd
import matplotlib.pyplot as plt 
import numpy as np

url = 'https://raw.githubusercontent.com/opengeos/leafmap/master/examples/data/countries.geojson'
gdf_countries = gpd.read_file(url)
gdf_countries.head(10)


In [ ]:
gdf_filter_Colombia =  gdf_countries.loc[gdf_countries["NAME"] == "Colombia"]
gdf_filter_Colombia

In [ ]:
gdf_colombia = gdf_filter_Colombia.to_crs(epsg=3116) #to find centroid we need to reproject first
gdf_colombia["centroid"] = gdf_filter_Colombia.centroid 

In [ ]:
ax = gdf_filter_Colombia.plot(figsize=(10, 4), edgecolor="black", alpha=0.6, color="green")
gdf_colombia["centroid"].plot(ax=ax, color="red", markersize=50)


In [ ]:
''#? Perform a spatial join between two GeoDataFrames: one containing polygons (e.g., country borders) and one 
#? containing points (e.g., cities). Find out which points fall within which polygons.

url = 'https://raw.githubusercontent.com/opengeos/leafmap/refs/heads/master/docs/data/world_cities.geojson'
gdf_cities = gpd.read_file(url)
gdf_cities.head()

In [ ]:
gdf_cities.crs

In [ ]:
# filtered continent south america based on gdf countries exercise 2
gdf_south_america = gdf_countries.loc[gdf_countries["CONTINENT"] == "South America"]
gdf_south_america.head(2)


In [ ]:
#renaming to keep both geometries cities and countries
gdf_south_america.rename(columns={"geometry": "geometry_country"}, inplace=True)
gdf_south_america = gdf_south_america.set_geometry('geometry_country')

In [ ]:
gdf_south_america.plot(figsize=(8, 3), legend=True)

In [ ]:
gdf_merged = gpd.sjoin(gdf_cities, gdf_south_america, how="inner")
gdf_merged

In [ ]:
gdf_merged.plot(figsize=(5, 2), legend=True)

In [ ]:
gdf_merged_col = gdf_merged.loc[gdf_merged["NAME"] == "Colombia"]
gdf_merged_col



In [ ]:
ax = gdf_filter_Colombia.plot(figsize=(8, 4), edgecolor="black", alpha=0.6, color="green")
gdf_merged_col["geometry"].plot(ax=ax, color="red", markersize=50)

In [ ]:
''# ? Exercise 4 -  proportions of the population of each city that falls within the country borders

gdf_south_america["population_log10"] = np.log10(gdf_south_america["POP_EST"])
gdf_south_america.plot("population_log10", legend=True, figsize=(8, 4))

In [ ]:
# Plot the data with the base-10 logarithmic transformation for visualization GPT
fig, ax = plt.subplots(1, 1, figsize=(20, 15))
gdf_south_america.plot(ax=ax, color='white', edgecolor='black')
gdf_south_america.plot(column="population_log10", ax=ax, legend=True, cmap='RdYlGn', markersize=50)

# Annotate the real population values
for x, y, label in zip(gdf_south_america.geometry.centroid.x, gdf_south_america.geometry.centroid.y, gdf_south_america["POP_EST"]):
    ax.text(x, y, f'{label:,}', fontsize=8, ha ='center' )

plt.title("Base-10 Logarithmic Display of Population Proportions with Real Population Values")
plt.show()

## Summary

This lecture provided an introduction to working with geospatial data using GeoPandas. We covered basic concepts such as reading/writing geospatial data, performing spatial operations (e.g., buffering, intersections), and visualizing geospatial data using maps. GeoPandas, built on Pandas and Shapely, enables efficient and intuitive geospatial analysis in Python.

In [69]:
import nbformat
import re

def extract_titles(notebook_path):
    # Load the notebook
    with open(notebook_path, 'r', encoding='utf-8') as f:
        nb = nbformat.read(f, as_version=4)
    
    titles = []
    
    # Regular expressions for HTML headings
    html_heading_re = re.compile(r'<h([1-6])>(.*?)</h\1>', re.IGNORECASE)
    
    # Iterate through the cells and extract titles
    for cell in nb.cells:
        if cell.cell_type == 'markdown':
            lines = cell.source.split('\n')
            for line in lines:
                if line.startswith('#'):
                    titles.append(line)
                else:
                    # Check for HTML headings
                    match = html_heading_re.match(line)
                    if match:
                        level = int(match.group(1))
                        title_text = match.group(2).strip()
                        titles.append('#' * level + ' ' + title_text)
    
    return titles

def generate_toc(titles):
    toc = []
    for title in titles:
        # Count the number of leading '#' to determine the level
        level = title.count('#')
        # Remove leading '#' and strip leading/trailing whitespace
        title_text = title.lstrip('#').strip()
        # Create the TOC entry with indentation based on the level
        toc.append('  ' * (level - 1) + f'- {title_text}')
    
    return '\n'.join(toc)

def main():
    notebook_path = 'geopandas.ipynb'  # Replace with your notebook path
    titles = extract_titles(notebook_path)
    toc = generate_toc(titles)
    print(toc)

if __name__ == '__main__':
    main()

- GeoPandas
  - Introduction
  - Concepts
  - Installing and Importing GeoPandas
  - 1. Creating GeoDataFrames
  - 2. Reading and Writing Geospatial Data
    - Downloading from internet
  - 3. Simple Accessors and Methods
    - Measuring Area
    - Getting Polygon Boundaries and Centroids
    - Measuring Distance and filtering
    - Calculating Mean Distance
  - 4. Plotting Geospatial Data
    - Plotting the Area of Each Borough
    - Plotting Centroids and Boundaries
  - Geometry Manipulations
    - Buffering Geometries
    - Convex Hulls
  - Spatial Queries and Relations
    - Checking for Intersections
    - Checking for Containment
  - Projections and Coordinate Reference Systems (CRS)
    - Checking the CRS
    - Reprojecting to WGS84
  - Exercises
  - Summary
